In [26]:
import pandas as pd
import numpy as np
import math
import copy as cp
from sklearn import tree
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

train_df = pd.read_csv("./data/train_final.csv")
m_train = len(train_df)
test_df = pd.read_csv("./data/test_final.csv")
m_test = len(test_df)

# preprocess the data
- workclass, occupation, native.country has anomaly data '?'

In [21]:


categorical_attrib_values = { "workclass":['Private', 'Self-emp-not-inc', 'Self-emp-inc', 'Federal-gov', 'Local-gov', 'State-gov', 'Without-pay', 'Never-worked'],\
                 "education":['Bachelors', 'Some-college', '11th', 'HS-grad', 'Prof-school', 'Assoc-acdm', 'Assoc-voc', '9th', '7th-8th', '12th', 'Masters', '1st-4th', '10th', 'Doctorate', '5th-6th', 'Preschool'],\
                 "marital.status":['Married-civ-spouse', 'Divorced', 'Never-married', 'Separated', 'Widowed', 'Married-spouse-absent', 'Married-AF-spouse'],\
                 "occupation":['Tech-support', 'Craft-repair', 'Other-service', 'Sales', 'Exec-managerial', 'Prof-specialty', 'Handlers-cleaners', 'Machine-op-inspct', 'Adm-clerical', 'Farming-fishing', 'Transport-moving', 'Priv-house-serv', 'Protective-serv', 'Armed-Forces'],\
                 "relationship":['Wife', 'Own-child', 'Husband', 'Not-in-family', 'Other-relative', 'Unmarried'],\
                 "race":['White', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other', 'Black'],\
                 "sex":['Female', 'Male'],\
                 "native.country": ['United-States', 'Cambodia', 'England', 'Puerto-Rico', 'Canada', 'Germany', 'Outlying-US(Guam-USVI-etc)', 'India', 'Japan', 'Greece', 'South', 'China', 'Cuba', 'Iran', 'Honduras', 'Philippines', 'Italy', 'Poland', 'Jamaica', 'Vietnam', 'Mexico', 'Portugal', 'Ireland', 'France', 'Dominican-Republic', 'Laos', 'Ecuador', 'Taiwan', 'Haiti', 'Columbia', 'Hungary', 'Guatemala', 'Nicaragua', 'Scotland', 'Thailand', 'Yugoslavia', 'El-Salvador', 'Trinadad&Tobago', 'Peru', 'Hong', 'Holand-Netherlands']
                }
def convert_numerical_to_binary(df):
    for col_name in df.columns:
        if col_name!= "ID" and col_name not in categorical_attrib_values:
            df[col_name] = pd.to_numeric(df[col_name], errors='coerce')
            threashold = df[col_name].median()
            print("preprocessing numerical attribute:", col_name, "threashold:", threashold)
            df[col_name] = df[col_name].apply(lambda x: 0 if int(x) <= threashold else 1)

train_df_processed = train_df
convert_numerical_to_binary(train_df_processed)

train_mode_country = train_df_processed["native.country"].mode()[0]
train_mode_workclass = train_df_processed["workclass"].mode()[0]
train_mode_occupation = train_df_processed["occupation"].mode()[0]
train_df_processed["native.country"]= train_df_processed["native.country"].replace("Holand-Netherlands",train_mode_country, regex=True)
train_df_processed["native.country"] = train_df_processed["native.country"].apply(lambda x: train_mode_country if x=='?' else x)
train_df_processed["workclass"] = train_df_processed["workclass"].apply(lambda x: train_mode_workclass if x=='?' else x)
train_df_processed["occupation"] = train_df_processed["occupation"].apply(lambda x: train_mode_occupation if x=='?' else x)


test_df_processed = test_df
convert_numerical_to_binary(test_df_processed)

test_mode_country = test_df_processed["native.country"].mode()[0]
test_mode_workclass = test_df_processed["workclass"].mode()[0]
test_mode_occupation = test_df_processed["occupation"].mode()[0]
test_df_processed["native.country"]= test_df_processed["native.country"].replace("Holand-Netherlands",test_mode_country, regex=True)
test_df_processed["native.country"] = test_df_processed["native.country"].apply(lambda x: test_mode_country if x=='?' else x)
test_df_processed["workclass"] = test_df_processed["workclass"].apply(lambda x: test_mode_workclass if x=='?' else x)
test_df_processed["occupation"] = test_df_processed["occupation"].apply(lambda x: test_mode_occupation if x=='?' else x)

preprocessing numerical attribute: age threashold: 37.0
preprocessing numerical attribute: fnlwgt threashold: 177299.5
preprocessing numerical attribute: education.num threashold: 10.0
preprocessing numerical attribute: capital.gain threashold: 0.0
preprocessing numerical attribute: capital.loss threashold: 0.0
preprocessing numerical attribute: hours.per.week threashold: 40.0
preprocessing numerical attribute: income>50K threashold: 0.0
preprocessing numerical attribute: age threashold: 37.0
preprocessing numerical attribute: fnlwgt threashold: 179266.5
preprocessing numerical attribute: education.num threashold: 10.0
preprocessing numerical attribute: capital.gain threashold: 0.0
preprocessing numerical attribute: capital.loss threashold: 0.0
preprocessing numerical attribute: hours.per.week threashold: 40.0


# Fit a tree

In [22]:
features = list(train_df.columns[:14])
y = train_df["income>50K"]
X = pd.get_dummies(train_df_processed[features],drop_first=True)
dt = DecisionTreeClassifier(min_samples_split=20, random_state=99)
dt.fit(X, y)
preds = dt.predict(X)
incorrect = np.count_nonzero(preds-y)
print("train_accuracy", 1 - incorrect/X.shape[0])

train_accuracy 0.87748


# Prediction

In [23]:
test_features = list(test_df.columns[1:])
test_X = pd.get_dummies(test_df_processed[test_features],drop_first=True)
test_df_processed["Prediction"] = dt.predict(test_X)
result = test_df_processed[["ID","Prediction"]]
result.to_csv("sl-dt.csv", index=False)

# Regression Tree

In [24]:
dtr = DecisionTreeRegressor() 
dtr.fit(X, y)
test_df_processed["Prediction"] = dtr.predict(test_X)
result = test_df_processed[["ID","Prediction"]]
result.to_csv("sl-dtr.csv", index=False)

# Random Forest

In [25]:
regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(X, y)
test_df_processed["Prediction"] = regr.predict(test_X)
result = test_df_processed[["ID","Prediction"]]
result.to_csv("sl-rf.csv", index=False)

# GradientBoosting

In [27]:
reg = GradientBoostingRegressor(random_state=0)
reg.fit(X, y)
test_df_processed["Prediction"] = reg.predict(test_X)
result = test_df_processed[["ID","Prediction"]]
result.to_csv("sl-eb.csv", index=False)

# Gradient boosting with cross validation